In [1]:
import numpy as np
import pandas as pd
from netCDF4 import Dataset
import tqdm
import itertools

In [2]:
moorings = ['a','b','d']

In [3]:
### monthly mean radar freeboard at each mooring

datapath = '/Users/carmennab/Dropbox/alpha_retracker/data/sit/pysiral/'

radar_freeboards_data = {}

for mooring in moorings:

    radar_freeboards_data[mooring] = pd.read_csv(datapath+'mean_rf_mooring_'+mooring+'.csv')

In [4]:
c = 3.
cs = 2.4
rho_w = 1023.9

### make ERA5 SITS, MERRA2 SITS, JUT SITS

In [5]:
alphas = np.round(np.arange(0,1.01,0.01),decimals=2)
thresholds = np.arange(5,97.5,2.5)

for name in ['era5','merra2','jut']:
    for mooring in moorings:
        rfs = radar_freeboards_data[mooring]
        aux = pd.read_csv('../data/sit/pysiral/aux_'+mooring+'.csv')

        if name == 'jut':
            df_rho_i = aux['mean_ice_density_jut']
            df_hs = aux['mean_era5_snod']
            df_rho_s = aux['mean_era5_sden']
            
        else:
            df_rho_i = aux['mean_ice_density']
            df_hs = aux['mean_'+name+'_snod']
            df_rho_s = aux['mean_'+name+'_sden']
        
        df_buoy_thickness = aux['mean_buoy_thickness_'+name]

        for alpha in tqdm.tqdm(alphas):

            sits = []

            for month, year in itertools.product(set(rfs['month']),set(rfs['year'])):

                d = {'year':year,'month':month}

                buoy_thickness = df_buoy_thickness[(aux['month']==month) & (aux['year']==year)]

                freeboard = rfs[(rfs['month']==month) & (rfs['year']==year)]

                if (len(buoy_thickness)>0) & (len(freeboard)>0):

                    for threshold in thresholds:

                        rf = rfs['mean_radar_freeboard'][(rfs['month']==month) & (rfs['year']==year) & (rfs['threshold']==threshold)].iloc[0]

                        hs = df_hs[(aux['month']==month) & (aux['year']==year)].iloc[0]

                        rho_s = df_rho_s[(aux['month']==month) & (aux['year']==year)].iloc[0]
                        rho_i = df_rho_i[(aux['month']==month) & (aux['year']==year)].iloc[0]
                        ice_freeboard = rf + (((alpha * (c/cs)) -1)*hs)  # isobel thesis pg 114
                        sit = ((ice_freeboard*rho_w) + (hs*rho_s)) / (rho_w - rho_i)
                        d['mean_sit_'+str(threshold)] = sit

                    d['mean_buoy_thickness'] = buoy_thickness.iloc[0]

                    sits.append(d)

            sit_df = pd.DataFrame(sits)
            filepath = f'../data/sit/pysiral/alpha_sits_{name}/a{str(alpha)}_{mooring}.csv'
            sit_df.to_csv(filepath, na_rep=np.nan, index=False)

100%|█████████████████████████████████████████| 101/101 [01:39<00:00,  1.01it/s]


### make ERA5 * 0.5 and * 1.5 SITS

In [5]:
alphas = np.round(np.arange(0,1.01,0.01),decimals=2)
thresholds = np.arange(5,97.5,2.5)

for name in ['90','110']:
    for mooring in moorings:
        rfs = radar_freeboards_data[mooring]
        aux = pd.read_csv('../data/sit/pysiral/aux_'+mooring+'.csv')


        df_rho_i = aux['mean_ice_density']
        df_hs = aux['mean_era5_snod_'+name]
        df_rho_s = aux['mean_era5_sden']
        
        df_buoy_thickness = aux['mean_buoy_thickness_'+name]

        for alpha in tqdm.tqdm(alphas):

            sits = []

            for month, year in itertools.product(set(rfs['month']),set(rfs['year'])):

                d = {'year':year,'month':month}

                buoy_thickness = df_buoy_thickness[(aux['month']==month) & (aux['year']==year)]

                freeboard = rfs[(rfs['month']==month) & (rfs['year']==year)]

                if (len(buoy_thickness)>0) & (len(freeboard)>0):

                    for threshold in thresholds:

                        rf = rfs['mean_radar_freeboard'][(rfs['month']==month) & (rfs['year']==year) & (rfs['threshold']==threshold)].iloc[0]

                        hs = df_hs[(aux['month']==month) & (aux['year']==year)].iloc[0]

                        rho_s = df_rho_s[(aux['month']==month) & (aux['year']==year)].iloc[0]
                        rho_i = df_rho_i[(aux['month']==month) & (aux['year']==year)].iloc[0]
                        ice_freeboard = rf + (((alpha * (c/cs)) -1)*hs)  # isobel thesis pg 114
                        sit = ((ice_freeboard*rho_w) + (hs*rho_s)) / (rho_w - rho_i)
                        d['mean_sit_'+str(threshold)] = sit

                    d['mean_buoy_thickness'] = buoy_thickness.iloc[0]

                    sits.append(d)

            sit_df = pd.DataFrame(sits)
            filepath = f'../data/sit/pysiral/alpha_sits_{name}/a{str(alpha)}_{mooring}.csv'
            sit_df.to_csv(filepath, na_rep=np.nan, index=False)

100%|█████████████████████████████████████████| 101/101 [03:19<00:00,  1.97s/it]
